# **Fish Project**

## Import

In [54]:
# Import required packages 
import tensorflow as tf
import tensorflow_hub as hub
import image_modeling   # import image_modeling.py file
import datetime
# Load the TensorBoard notebook extension
%load_ext tensorboard

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [55]:
# Clear any logs from previous runs
!rm -rf ./logs/

In [ ]:
# Import variables from image_modelling.py file
HEIGHT = image_modeling.HEIGHT
WIDTH = image_modeling.WIDTH
NCLASSES = image_modeling.NCLASSES
CLASS_NAMES = image_modeling.CLASS_NAMES
BATCH_SIZE = image_modeling.BATCH_SIZE
#TRAINING_SIZE = !wc -l < Niels_file.csv
#TRAINING_STEPS = int(TRAINING_SIZE[0]) // BATCH_SIZE

#TRAIN_PATH = 'Niels_file.csv'
#EVAL_PATH = '???.csv'
#TEST_PATH = '???.csv'

In [ ]:
# You can compare this output with the variables in the image_modelling.py file...
print(HEIGHT)
print(CLASS_NAMES)
print(NCLASSES)

## Building our Model

Building and training a neural network involves various steps: 
1. define the architecture of the model
2. compile the model
3. train the model
4. evaluate the model

We have to start with defining the architecture. Our neural network will consist of several layers that are chained together. The input layer of our model will take our input data and hand it over to the flatten layer, which is responsible for reformatting our data. It will transform the format of our images from a three-dimensional array (HEIGHT, WIDTH, 3) to a one-dimensional array of size HEIGHT * WIDTH * 3. 
After the pixels are flattened we use a dense layer that returns a logits array with length `NCLASSES`. Each node in this layer contains a score that indicates the current image belongs to one of the n classes. 

### Simple Model

In [ ]:
# Lets create a simple linear model.
def linear_model():
    model = tf.keras.models.Sequential()
    model.add(tf.keras.layers.InputLayer(input_shape=[HEIGHT, WIDTH, 3], name='image'))
    model.add(tf.keras.layers.Flatten(data_format="channels_last"))
    # We want to have a simple linear model so we have 
    # no activation function. 
    model.add(tf.keras.layers.Dense(units=NCLASSES, activation=None))
    return model

Before we can train our model we need to compile it and define more settings. We have to choose a loss function, an optimizer and metrics. 
* The **loss function** measures how accurate the model is during training by calculating the model error. Usually we want to minimize this function to improve our model. As you can see in the [TensorFlow documentation](https://www.tensorflow.org/api_docs/python/tf/keras/losses) there are lot's of different loss functions to choose from. Some, e.g. the mean squared errror, hopefully look familiar to you. ;) 
* The **[optimizer](https://www.tensorflow.org/api_docs/python/tf/keras/optimizers)** defines how the model is updated based on the data and the loss function. One optimizer we've already covered earlier and which is also used for neural networks is the stochastic gradient descent (SGD) algorithm.   
* The **metric** is used to monitor the training process. Here we can choose one of the metrics we've already encountered or [many more](https://www.tensorflow.org/api_docs/python/tf/keras/metrics). 

The following function compiles our model, loads the data using the `load_dataset()` function from the image_modelling.py file and trains the model on the loaded data. In the end the function returns our fitted model. 

In [ ]:
# Create a function for training and evaluation
def train_and_evaluate(model, batch_size):
    
    model.compile(
        optimizer="adam", 
        # The model outputs one-hot-encoded logits, so we need
        # use the sparse version of the crossentropy loss.
        loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True),
        metrics=['accuracy']
    )
    
    dataset = image_modeling.load_dataset(TRAIN_PATH, batch_size)
    eval_dataset = image_modeling.load_dataset(EVAL_PATH, batch_size, training=False)
    
    log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
    tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

    model.fit(
        dataset, 
        validation_data=eval_dataset,
        steps_per_epoch=TRAINING_STEPS, 
        epochs=10,
        callbacks=[tensorboard_callback]
    )
    
    
    return model

In [ ]:
# Build and train our model using the prior defined functions 
model = linear_model()
trained_model = train_and_evaluate(model, BATCH_SIZE)

Let us use Tensorboard to monitor our results:

In [ ]:
%tensorboard --logdir logs/fit

Since we worked entirely with functions so far, we will also define one for testing our model. The function will load our test data and use it in order to evaluate the performance of our model on unseen data.

In [ ]:
# Write a testing function.
def test(model):
    
    test_dataset = image_modeling.load_dataset(TEST_PATH, batch_size=1, training=False)
    model.evaluate(test_dataset)

In [ ]:
# Call the testing function for our model
test(model)

### Deep Neural Network

Our simple model is not performing well. Maybe we can boost its performance by adding more layers.

In the following `dnn_model()` function we add three more hidden, dense layers after the flatten layer to increase our models complexity. 

In [ ]:
# Lets compare a neural network with hidden layers to the linear model
def dnn_model():
    model = tf.keras.models.Sequential()
    model.add(tf.keras.layers.InputLayer(input_shape=[HEIGHT, WIDTH, 3], name='image'))
    model.add(tf.keras.layers.Flatten(data_format="channels_last"))
    model.add(tf.keras.layers.Dense(units = 40, activation = "relu"))
    model.add(tf.keras.layers.Dense(units = 40, activation = "relu"))
    model.add(tf.keras.layers.Dense(units = 30, activation = "relu"))
    # We want to have a simple linear model so we have 
    # no activation function. 
    model.add(tf.keras.layers.Dense(units=NCLASSES, activation=None))
    return model

In [ ]:
# Let us fit the deep neural network
model = dnn_model()
trained_model = train_and_evaluate(model, BATCH_SIZE)

In [ ]:
test(model)

Adding more hidden layers to our model, did indeed increase the accuracy. But still, the model's performance leaves something to be desired. Since we are working with images, switching to a convolutional neural network might help.

### Convolutional Neural Network

CNN's are widely used for image recognition. They are regularized versions of DNN's able to be deeper without generating as much parameters due to its [convolutional](https://machinelearningmastery.com/convolutional-layers-for-deep-learning-neural-networks/) and pooling layers.

The architecture of our CNN is even more complex. This time we combine dense layers with `Conv2D` and `MaxPooling2D` layers. The convolutional and max pooling layers are inserted between the input layer and the flatten layer. 

In [ ]:
# Now let us move on to a CNN model. 
def cnn_model():
    model = tf.keras.models.Sequential()
    model.add(tf.keras.layers.InputLayer(input_shape=[HEIGHT, WIDTH, 3], name='image'))
    model.add(tf.keras.layers.Conv2D(filters=10, kernel_size=[5, 5], padding="same", activation="relu"))
    model.add(tf.keras.layers.MaxPooling2D(pool_size=[2, 2], strides=2))
    model.add(tf.keras.layers.Conv2D(filters=20, kernel_size=[5, 5], padding="same", activation="relu"))
    model.add(tf.keras.layers.MaxPooling2D(pool_size=[2, 2], strides=2))
    model.add(tf.keras.layers.Flatten())
    model.add(tf.keras.layers.Dense(units=300, activation="relu"))
    model.add(tf.keras.layers.Dense(units=NCLASSES, activation=None))
    return model

We can have a look at the architecture of our model with the method `.summary`. As you can see in the summary below, the output of each `Conv2D` and `MaxPooling2D` layer is also a three dimensional tensor of shape (height, width, channels). As we go deeper into the network the dimensions shrink. One advantage of the shrinking dimensions is that we can computationally afford to add more output channels in each convolutional layer. We can control the number of the output channels of those layers with the `filters` argument. 

However, at the end of our model we still need the combination of the flatten and dense layers to perform classification. 

In [ ]:
model = cnn_model()
model.summary()

In [ ]:
# Let us fit the convolutional neural network
model = cnn_model()
trained_model = train_and_evaluate(model, BATCH_SIZE)

In [ ]:
test(model)

We see the CNN give better results than the DNN. But we still have heavy overfitting.

>__Exercise__: try to reduce overfitting of your model by: 
- making full use of your data augmentation by increasing `steps_per_epoch` in the `train_and_evaluate` function.
- [adding regularization](https://keras.io/api/layers/regularizers/) to the `Conv2D` and `Dense` layers.
- Adding dropout layers. See section CNN Dropout Regularization of [this](https://machinelearningmastery.com/how-to-reduce-overfitting-with-dropout-regularization-in-keras/)

You can also try adding additional `Conv2D`, `MaxPooling2D` or `Dense` layers.

## Transfer Learning

Transfer learning is when a model is trained on one task and is then reused for another task. One approach to transfer learning is fine-tuning. Here you take a trained neural net, exchange the last layer (head) for another layer, that fits the new task and then train the weights of the last layer only. 

First we need to download the headless model (this can take a while) we use MobileNetV2 which is a CNN that was trained on the [ImageNet](https://en.wikipedia.org/wiki/ImageNet) dataset, consisting of over 14 million images:

In [ ]:
feature_extractor_url = "https://tfhub.dev/google/tf2-preview/mobilenet_v2/feature_vector/2"
feature_extractor_layer = hub.KerasLayer(feature_extractor_url,input_shape=(HEIGHT,WIDTH,3))

We only want to train the last layer therefore we freeze the layers of our headless model:

In [ ]:
feature_extractor_layer.trainable = False

Now we define our model by simply adding the output layer to our pretrained net.

In [ ]:
def transfer_learning_model():
    model = tf.keras.models.Sequential()
    model.add(feature_extractor_layer)
    # TODO: add the correct output layer here
    model.add(tf.keras.layers.Dense(units=NCLASSES, activation=None))
    return model

In [ ]:
# Let us fit our transfer learning model
model = transfer_learning_model()
trained_model = train_and_evaluate(model, BATCH_SIZE)

In [ ]:
test(model)

As we see the results of fine-tuning surpass the results of the linear model, DNN and CNN. Fine-tuning is a very powerful approach which can generalize well even with limited amount of data.

### Summary 

You trained four different model architectures linear, DNN, CNN, and fine-tuned MobileNetV2. With increasing complexity of the model (number of parameters) the results get better but also overfitting is more of an issue. Because with lots of parameters and not so much images the NN can memorize each image in the train set without learning more general features. You tried out different ways to counteract overfitting:
- Training on more data. Data augmentation is a way to generate more data and therefore helps. 
- Using a CNN architecture. The CNN's convolutional and max pooling layers reduce the number of parameters allowing the network to be deeper. Leading to less overfitting.
- L1 or L2 Regularization is another possibility to deal with overfitting.
- Adding dropout layers. Helps prevent [co-adapting](https://machinelearning.wtf/terms/co-adaptation/) and overfitting.

Finally you used a pretrained CNN with fine-tuning to utilize the power of a network trained on over 14 million images. 
